In [16]:
import pprint
import numpy as np
import pandas as pd
import sys
import re
import itertools
import copy
# sys.path.append("./recipes/recipes/lib/python3.8/site-packages")
import pymongo 
import glob
from pymongo import MongoClient
with open(glob.glob('/home/*/*/*/mymongo')[0]) as f:
    pass_key = f.read()
pass_key=pass_key.strip('\n')

In [17]:
user_name = 'ze_readOnly'
from pymongo.errors import ConnectionFailure
client = MongoClient()
try:
    # client.admin.command('ping')
    client = MongoClient(username=user_name, password=pass_key)
    print('Server available, you are authenticated as user %s.'%(user_name))

except ConnectionFailure:
    print("Server not available")

Server available, you are authenticated as user ze_readOnly.


In [18]:
db = client.allrecipes
r = db.recipes

In [19]:
# snippet taken and adapted  from https://thispointer.com/python-how-to-add-append-key-value-pairs-in-dictionary-using-dict-update/
def append_value(dict_obj, key, value):
    [dict_obj[k].append(v) for k,v in zip(key,value)]
# for speed vectorize above function
append_value_v = np.vectorize(append_value)

In [21]:
# append all dicts of recipes from one Author
reciped = {'_id': [], 'title': [], 'Author': [], 'ingredient': [], 'direction': [], 
           'nutrition_per_serving': [], 'total_ratings': [], 
           'ratings': [], 'prep': [], 'cook': [], 'additional': [], 
           'total': [], 'Servings': [], 'Yield': []
          }

keys = list(reciped.keys())
for recipe in r.find({"Author": 'MakeItHealthy' }):
    values = [recipe.get(k) for k in keys]
    append_value(reciped,keys,values)
len(reciped)

14

In [23]:
# create a pandas data frame
allkeys = list(reciped.keys())
df = pd.DataFrame([reciped[k] for k in keys]).T
df.columns = allkeys

In [24]:
df.head()

,_id,title,Author,ingredient,direction,nutrition_per_serving,total_ratings,ratings,prep,cook,additional,total,Servings,Yield
0,611c269728708657c245a3d9,"Healthier Best Big, Fat, Chewy Chocolate Chip ...",MakeItHealthy,"[2 cups all-purpose flour , ½ teaspoon baking ...","[[Step 1, Preheat oven to 325 degrees F (165 d...",[232 calories; protein 2.6g; carbohydrates 38g...,[5 Ratings],"{'5': 3, '4': 0, '3': 0, '2': 0, '1': 2}",10 mins,15 mins,1 hr,1 hr 25 mins,18,1 1/2 dozen
1,611c273428708657c245a403,Healthier (but still) The Best Rolled Sugar Co...,MakeItHealthy,"[1 ½ cups butter, softened , 1 cup granular su...","[[Step 1, Cream together butter, sweetener, an...",[88 calories; protein 1.5g; carbohydrates 9.3g...,[3 Ratings],"{'5': 1, '4': 2, '3': 0, '2': 0, '1': 0}",20 mins,8 mins,1 hr,1 hr 28 mins,60,5 dozen cookies
2,611c27d028708657c245a42f,Healthier Guacamole,MakeItHealthy,"[¾ cup crumbled tofu , 2 avocados - peeled and...","[[Step 1, Place tofu and one avocado in a food...",[214 calories; protein 6.4g; carbohydrates 13....,[2 Ratings],"{'5': 1, '4': 0, '3': 0, '2': 1, '1': 0}",10 mins,None,None,10 mins,4,4 servings
3,611c2b1128708657c245a511,Healthier Sweet Potato Pie I,MakeItHealthy,"[1 (1 pound) sweet potato , ½ cup butter, soft...","[[Step 1, Boil sweet potato whole in skin in a...",[316 calories; protein 4.3g; carbohydrates 33....,[3 Ratings],"{'5': 3, '4': 0, '3': 0, '2': 0, '1': 0}",30 mins,50 mins,1 hr,2 hrs 20 mins,8,1 9-inch pan
4,611c2bc428708657c245a542,Healthier Best Chocolate Chip Cookies,MakeItHealthy,"[½ cup butter, softened , ¼ cup sweet potato p...","[[Step 1, Preheat oven to 350 degrees F (175 d...",[249 calories; protein 3.7g; carbohydrates 35....,[3 Ratings],"{'5': 2, '4': 1, '3': 0, '2': 0, '1': 0}",20 mins,10 mins,30 mins,1 hr,24,4 dozen


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   _id                    40 non-null     object
 1   title                  40 non-null     object
 2   Author                 40 non-null     object
 3   ingredient             40 non-null     object
 4   direction              40 non-null     object
 5   nutrition_per_serving  40 non-null     object
 6   total_ratings          40 non-null     object
 7   ratings                40 non-null     object
 8   prep                   40 non-null     object
 9   cook                   35 non-null     object
 10  additional             23 non-null     object
 11  total                  40 non-null     object
 12  Servings               40 non-null     object
 13  Yield                  40 non-null     object
dtypes: object(14)
memory usage: 4.5+ KB


In [26]:
# working on a copy is always safer with mutable data sturcutres  like pandas data frames
dff = copy.deepcopy(df)

In [27]:
# print out cooking directions
for ttl, dr in zip(dff['title'], dff['direction']):
    direction = list(itertools.chain(*dr))
    ds = [d for d in direction  if d != None ]
    print('\x1b[1;31m'+ttl+'\x1b[0m')
    print("\n".join(ds[:-1]))

Healthier Best Big, Fat, Chewy Chocolate Chip Cookie
Step 1
Preheat oven to 325 degrees F (165 degrees C). Grease baking sheets or line with parchment paper.
Step 2
Sift together flour and baking soda in a small bowl; set aside.
Step 3
Beat melted butter, yogurt, brown sugar, and white sugar in a bowl until blended. Beat in vanilla extract, egg, and egg yolk until light and creamy; mix in flour mixture until just blended. Stir in chocolate chips, blueberries, and cranberries by hand using a wooden spoon. Drop 1/4 cup cookie dough 3-inches apart on prepared baking sheets.
Step 4
Bake in preheated oven until edges are lightly toasted, 15 to 17 minutes. Cool on baking sheets for 5 minutes before transferring to wire racks to cool completely.
This recipe is a healthier version of 
Healthier (but still) The Best Rolled Sugar Cookies
Step 1
Cream together butter, sweetener, and sugar in a large bowl, until smooth. Beat in eggs and vanilla extract. Stir in flour and baking powder. Cover and c

In [ ]:
# nut_list = list(itertools.chain(*dff['nutrition_per_serving']))
# nut = nut_list[0].replace(';','')
# nut = nut.split(' ')
# nut[0],nut[1] = nut[1],nut[0]
# nut


In [28]:
def fun_int(nut):
    # takes a list, nut
    # extracts the numeric parts of nut units
    # returns  nut values and units
    nut_names = [name for name in nut[::2]]
    values = []
    units = []
    for n in nut:
        try:
            value = '.'.join(re.findall(r'\d+',n))
            values.append(float(value))

            if isinstance (float(value), float):
                unit = n.strip(value)
                units.append(unit)
#             print('value:',value,'unit:',unit)
        except ValueError:
            pass
        
    return nut_names, values, units 

# fun_int_v = np.vectorize(fun_int)
# nut_names,values,units = fun_int(nut)
# # construct a data frame of nut values and units
# dnut = pd.DataFrame({'nut_value':values, 'nut_unit': units},index=nut_names)

# dnut

In [29]:
# As noted above some columns are made of lists. Need the  nutrition_per_serving in particular and going to flesh the data out from inside the lists

nut_list = list(itertools.chain(*dff['nutrition_per_serving'])) # gets a list out of data frame

# creating dictionaries 
Nut_names = []
Values = []
Units = []
ObjId = []
    
for nuts,objid in zip(nut_list,dff['_id']):

    nut = nuts.replace(';','')
    nut = nut.split(' ')
    nut[0], nut[1] = nut[1], nut[0]

    nut_names,values,units = fun_int(nut)
    Nut_names.append(nut_names)
    Values.append(values)
    Units.append(units)
    ObjId.append([str(objid)]*len(nut_names))


In [30]:
list(itertools.chain(*Nut_names))
list(itertools.chain(*Values))
list(itertools.chain(*Units))

arrays = [ np.array(list(itertools.chain(*ObjId))),
          np.array(list(itertools.chain(*Nut_names)))]
dnut = pd.DataFrame({
                     'nut_value':list(itertools.chain(*Values)), 
                     'nut_unit': list(itertools.chain(*Units)),
                    },index = arrays)
dnut = dnut.astype({'nut_value': 'float64'}).T
dnut

611c269728708657c245a3d9                                         \
                          calories protein carbohydrates  fat cholesterol   
nut_value                    232.0     2.6          38.0  8.7        35.3   
nut_unit                                 g             g    g          mg   

                 611c273428708657c245a403                             ...  \
          sodium                 calories protein carbohydrates  fat  ...   
nut_value   46.7                     88.0     1.5           9.3  5.0  ...   
nut_unit      mg                                g             g    g  ...   

          611c666a28708657c245b55e                                        \
                           protein carbohydrates  fat cholesterol sodium   
nut_value                      2.0          16.2  5.9        23.9  109.3   
nut_unit                         g             g    g          mg     mg   

          611c690728708657c245b614                                    
                          calories protein carbohydrates  fat sodium  
nut_value                    235.0     6.7          62.4  1.3  346.1  
nut_unit                                 g             g    g     mg  

[2 rows x 238 columns]

In [31]:
# parameters collected from various sources including USDA, https://www.healthline.com/health/high-cholesterol/rda, and https://www.acefitness.org/education-and-resources/professional/expert-articles/5904/how-to-determine-the-best-macronutrient-ratio-for-your-goals/
CARB = np.array([.45, .65])  # calorie fraction from carbohydrate 
PRO = np.array([.1,.35])    # calorie fraction from protien
FAT = np.array([.2, .35])   # calorie fraction from fat
CHOL = np.array([5.5, 6.5]) # This corresponds to min and max of cholesterol ([11000/2000, 13000/2000])  mg/cal min and max of cholesterol 
SOD = np.array([1, 2])    # min and max of Sodium

# Pretend we are food experts... and rate food like so
def food_rater(dff):
    df = copy.deepcopy(dff)
    
    if 'calories' in df:
        tot_cal = df['calories'].get('nut_value')
    else:
        if 'cholesterol' not in df:
            tot_cal = 4.0*df['carbohydrates'].get('nut_value') +4.0*df['protein'].get('nut_value') + 9.0*df['fat'].get('nut_value') 

        else:
            tot_cal = 4.0*df['carbohydrates'].get('nut_value') +4.0*df['protein'].get('nut_value') +9.0*df['fat'].get('nut_value') +0.009*df['cholesterol'].get('nut_value') # assuming unit is mg and cholestrol is still fat with 9 cal/gram cal source

            
                    
    Xcarb = 4.0*df['carbohydrates'].get('nut_value')/tot_cal
    Xpro = 4.0*df['protein'].get('nut_value')/tot_cal
    Xfat = 9.0*df['fat'].get('nut_value')/tot_cal
    Xsod = df['sodium'].get('nut_value')/tot_cal

    if 'cholesterol' in df:
        Xchol = df['cholesterol'].get('nut_value')/tot_cal
        if Xchol <= CHOL[0]:
            df.loc['nut_value','cholesterol_level'] = 'low'
        elif Xchol >= CHOL[1]:
            df.loc['nut_value','cholesterol_level'] = 'high'
        else:
            df.loc['nut_value','cholesterol_level'] = 'normal'
    
    
    if Xcarb  <= CARB[0]:
        df.loc['nut_value','carbohydrates_level'] = 'low'
        df.loc['nut_value','carbohydrates_score'] = 1
    elif Xcarb >= CARB[1]:
        df.loc['nut_value','carbohydrates_level'] = 'high'
        df.loc['nut_value','carbohydrates_score'] = 0
    else:
        df.loc['nut_value','carbohydrates_level'] = 'normal' 
        df.loc['nut_value','carbohydrates_score'] = 2
        
    if Xpro  <= PRO[0]:
        df.loc['nut_value','protien_level'] = 'low'
        df.loc['nut_value','protien_score'] = 1
    elif Xpro >= PRO[1]:
        df.loc['nut_value','protien_level'] = 'high'
        df.loc['nut_value','protien_score'] = 0
    else:
        df.loc['nut_value','protien_level'] = 'normal' 
        df.loc['nut_value','protien_score'] = 2
        
    if Xfat  <= FAT[0]:
        df.loc['nut_value','fat_level'] = 'low'
        df.loc['nut_value','fat_score'] = 1
    elif Xfat >= FAT[1]:
        df.loc['nut_value','fat_level'] = 'high'
        df.loc['nut_value','fat_score'] = 0
    else:
        df.loc['nut_value','fat_level'] = 'normal'
        df.loc['nut_value','fat_score'] = 2
        

    if Xsod <= 1: 
        df.loc['nut_value','sodium_level'] = 'good' 
        df.loc['nut_value','sodium_score'] = 2
    elif Xsod >= SOD[0] and Xsod <=SOD[1]:
        df.loc['nut_value','sodium_level'] = 'normal' 
        df.loc['nut_value','sodium_score'] = 1
    else:  
        df.loc['nut_value','sodium_level'] = 'bad' 
        df.loc['nut_value','sodium_score'] = 0
        
    tot_score = df.loc['nut_value',['_score' in col for col in  df.columns]].sum()
    if tot_score in [0,2]:
         df['overall_food_quality'] = 'bad'
    elif tot_score in [3,4]:
         df['overall_food_quality'] = 'good'
    else:
         df['overall_food_quality'] = 'excellent'
    return df
    

In [32]:
# Now rate food based on nutrient composition. Note that the food id is appended to each food so we can trace back to it later on
food=pd.DataFrame()
for ids in dff['_id'].values:
    tot_cal = food_rater(dnut.get(str(ids)))
    tot_cal['food_id'] = ids
    food = food.append(tot_cal)
food.loc['nut_value',:].head()

,calories,protein,carbohydrates,fat,cholesterol,sodium,cholesterol_level,carbohydrates_level,carbohydrates_score,protien_level,protien_score,fat_level,fat_score,sodium_level,sodium_score,overall_food_quality,food_id
nut_value,232.0,2.6,38.0,8.7,35.3,46.7,low,high,0.0,low,1.0,normal,2.0,good,2.0,excellent,611c269728708657c245a3d9
nut_value,88.0,1.5,9.3,5.0,24.6,53.8,low,low,1.0,low,1.0,high,0.0,good,2.0,good,611c273428708657c245a403
nut_value,214.0,6.4,13.7,17.1,NaN,595.3,NaN,low,1.0,normal,2.0,high,0.0,bad,0.0,good,611c27d028708657c245a42f
nut_value,316.0,4.3,33.6,18.6,79.4,245.3,low,low,1.0,low,1.0,high,0.0,good,2.0,good,611c2b1128708657c245a511
nut_value,249.0,3.7,35.8,11.4,10.2,95.5,low,normal,2.0,low,1.0,high,0.0,good,2.0,excellent,611c2bc428708657c245a542


In [33]:
# try to calculate total labour cost assuming a minimium wage (give yourself a higher wage if you are a pro chief ... or whatever)
labour = copy.deepcopy(dff[["prep","cook","additional","total"]])
labour.head()

,prep,cook,additional,total
0,10 mins,15 mins,1 hr,1 hr 25 mins
1,20 mins,8 mins,1 hr,1 hr 28 mins
2,10 mins,None,None,10 mins
3,30 mins,50 mins,1 hr,2 hrs 20 mins
4,20 mins,10 mins,30 mins,1 hr


In [34]:
# as you can see above we need to clean up and homogenize the time units. We do that below and compute the total labour cost for each food recipe
def homo_time_units(x):
    if x is not None:
        x = x.replace('s','')
        x_list = x.split(' ')

        if 'hr' in x_list and 'min' in x_list:
            y =float(x_list[0]) + float(x_list[2])/60.0 # time in units of hr
        else:        
            if 'min' in x:
                y=float(x.replace('min',''))/60.0 # time in units of hr

            if  'hr' in x:
                y = float(x.replace('hr','')) 
                
        return  y 

labour['tot_hrs'] = labour['total'].apply(homo_time_units)
labour['tot_labour_cost($)']= labour['tot_hrs'].apply(lambda x: 10*x)
labour['_id'] = dff['_id']
labour.head()


,prep,cook,additional,total,tot_hrs,tot_labour_cost($),_id
0,10 mins,15 mins,1 hr,1 hr 25 mins,1.416667,14.166667,611c269728708657c245a3d9
1,20 mins,8 mins,1 hr,1 hr 28 mins,1.466667,14.666667,611c273428708657c245a403
2,10 mins,None,None,10 mins,0.166667,1.666667,611c27d028708657c245a42f
3,30 mins,50 mins,1 hr,2 hrs 20 mins,2.333333,23.333333,611c2b1128708657c245a511
4,20 mins,10 mins,30 mins,1 hr,1.000000,10.000000,611c2bc428708657c245a542


In [40]:
# wish I could do something with the ingredients so that I can estimate the material cost of each recipe. That would be great. Main problem is getting/scraping prices of each ingredients. Tried walmart's page... but they didn't like it. Told to beat it. ( also read their robots.txt file) 
# Anyway, going to look into the data here a bit further...
print('Ingredients for "%s" recipe:'%(dff.loc[0,'title']))
dff['ingredient'][0]

Ingredients for "Healthier Best Big, Fat, Chewy Chocolate Chip Cookie" recipe:


['2 cups all-purpose flour ',
 '½ teaspoon baking soda ',
 '½ cup unsalted butter, melted ',
 '¼ cup non-fat plain yogurt ',
 '1 cup packed brown sugar ',
 '½ cup white sugar ',
 '1 tablespoon vanilla extract ',
 '1 egg ',
 '1 egg yolk ',
 '1 cup semisweet chocolate chips ',
 '½ cup blueberries ',
 '½ cup dried cranberries ']

In [41]:
# Umm...the 20 unicodes at it again. No expert on html ( no expert on anything for that matter - just a dude with too many interestes than he can work on)
# I can do some silly coding to work around them as

stupid_dict={'¼':.250, 
 '½':.500, 
 '¾':.750,
 '⅐':.142,  
'⅑':.111,  
'⅒':.010, 
 '⅓':.333,  
 '⅔':.667, 
 '⅕':.200, 
 '⅖':.400,  
 '⅗':.600,  
 '⅘':.800,  
 '⅙':.167,  
 '⅚':833,  
 '⅛':.125, 
 '⅜':.375,  
 '⅝':.675,  
 '⅞':.875,  
 '⅟':1.00,  
 '↉':0.00} 

for ks in dff['ingredient'][0]:

    [print('{}\n{}'.format(s,ks)) for s in stupid_dict.keys() if s in  ks]

½
½ teaspoon baking soda 
½
½ cup unsalted butter, melted 
¼
¼ cup non-fat plain yogurt 
½
½ cup white sugar 
½
½ cup blueberries 
½
½ cup dried cranberries 


In [42]:
# It's all about ratings. Let's see how each recipe does on a weighted average sense. Use the weight w = [5,4,3,2,1] corresponding to the number of stars and the order they are in the data
# calculate weighted means of star ratings of each recipe
def wtavg_star(r):
    w = np.array([5,4,3,2,1])
    avg_star = np.mean(list(r.values())*w)/np.sum(w)
    return avg_star 
dff['weighted_mean_ratings'] = dff['ratings'].apply(wtavg_star)
dff.head()

,_id,title,Author,ingredient,direction,nutrition_per_serving,total_ratings,ratings,prep,cook,additional,total,Servings,Yield,weighted_mean_ratings
0,611c269728708657c245a3d9,"Healthier Best Big, Fat, Chewy Chocolate Chip ...",MakeItHealthy,"[2 cups all-purpose flour , ½ teaspoon baking ...","[[Step 1, Preheat oven to 325 degrees F (165 d...",[232 calories; protein 2.6g; carbohydrates 38g...,[5 Ratings],"{'5': 3, '4': 0, '3': 0, '2': 0, '1': 2}",10 mins,15 mins,1 hr,1 hr 25 mins,18,1 1/2 dozen,0.226667
1,611c273428708657c245a403,Healthier (but still) The Best Rolled Sugar Co...,MakeItHealthy,"[1 ½ cups butter, softened , 1 cup granular su...","[[Step 1, Cream together butter, sweetener, an...",[88 calories; protein 1.5g; carbohydrates 9.3g...,[3 Ratings],"{'5': 1, '4': 2, '3': 0, '2': 0, '1': 0}",20 mins,8 mins,1 hr,1 hr 28 mins,60,5 dozen cookies,0.173333
2,611c27d028708657c245a42f,Healthier Guacamole,MakeItHealthy,"[¾ cup crumbled tofu , 2 avocados - peeled and...","[[Step 1, Place tofu and one avocado in a food...",[214 calories; protein 6.4g; carbohydrates 13....,[2 Ratings],"{'5': 1, '4': 0, '3': 0, '2': 1, '1': 0}",10 mins,None,None,10 mins,4,4 servings,0.093333
3,611c2b1128708657c245a511,Healthier Sweet Potato Pie I,MakeItHealthy,"[1 (1 pound) sweet potato , ½ cup butter, soft...","[[Step 1, Boil sweet potato whole in skin in a...",[316 calories; protein 4.3g; carbohydrates 33....,[3 Ratings],"{'5': 3, '4': 0, '3': 0, '2': 0, '1': 0}",30 mins,50 mins,1 hr,2 hrs 20 mins,8,1 9-inch pan,0.200000
4,611c2bc428708657c245a542,Healthier Best Chocolate Chip Cookies,MakeItHealthy,"[½ cup butter, softened , ¼ cup sweet potato p...","[[Step 1, Preheat oven to 350 degrees F (175 d...",[249 calories; protein 3.7g; carbohydrates 35....,[3 Ratings],"{'5': 2, '4': 1, '3': 0, '2': 0, '1': 0}",20 mins,10 mins,30 mins,1 hr,24,4 dozen,0.186667


<br>Enough with the ratings, now merge all new data columns created</br>
  They are:
 1. "tot_labour_cost($)" from labour df
 2. all columns from food df. For merging to work smoothly, update the column name food_id to simply _id. Ignore nut_unit rows at this point

In [43]:
food.rename(columns={'food_id':'_id'},inplace=True)
food.head()

,calories,protein,carbohydrates,fat,cholesterol,sodium,cholesterol_level,carbohydrates_level,carbohydrates_score,protien_level,protien_score,fat_level,fat_score,sodium_level,sodium_score,overall_food_quality,_id
nut_value,232.0,2.6,38.0,8.7,35.3,46.7,low,high,0.0,low,1.0,normal,2.0,good,2.0,excellent,611c269728708657c245a3d9
nut_unit,,g,g,g,mg,mg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,excellent,611c269728708657c245a3d9
nut_value,88.0,1.5,9.3,5.0,24.6,53.8,low,low,1.0,low,1.0,high,0.0,good,2.0,good,611c273428708657c245a403
nut_unit,,g,g,g,mg,mg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,good,611c273428708657c245a403
nut_value,214.0,6.4,13.7,17.1,NaN,595.3,NaN,low,1.0,normal,2.0,high,0.0,bad,0.0,good,611c27d028708657c245a42f


In [45]:
labour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   prep                40 non-null     object 
 1   cook                35 non-null     object 
 2   additional          23 non-null     object 
 3   total               40 non-null     object 
 4   tot_hrs             40 non-null     float64
 5   tot_labour_cost($)  40 non-null     float64
 6   _id                 40 non-null     object 
dtypes: float64(2), object(5)
memory usage: 2.3+ KB


In [46]:
food_quality = copy.deepcopy(food.loc['nut_value',:].reset_index().drop(columns=['index']))
food_quality.head()

,calories,protein,carbohydrates,fat,cholesterol,sodium,cholesterol_level,carbohydrates_level,carbohydrates_score,protien_level,protien_score,fat_level,fat_score,sodium_level,sodium_score,overall_food_quality,_id
0,232.0,2.6,38.0,8.7,35.3,46.7,low,high,0.0,low,1.0,normal,2.0,good,2.0,excellent,611c269728708657c245a3d9
1,88.0,1.5,9.3,5.0,24.6,53.8,low,low,1.0,low,1.0,high,0.0,good,2.0,good,611c273428708657c245a403
2,214.0,6.4,13.7,17.1,NaN,595.3,NaN,low,1.0,normal,2.0,high,0.0,bad,0.0,good,611c27d028708657c245a42f
3,316.0,4.3,33.6,18.6,79.4,245.3,low,low,1.0,low,1.0,high,0.0,good,2.0,good,611c2b1128708657c245a511
4,249.0,3.7,35.8,11.4,10.2,95.5,low,normal,2.0,low,1.0,high,0.0,good,2.0,excellent,611c2bc428708657c245a542


In [47]:
food_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   calories              40 non-null     object 
 1   protein               40 non-null     object 
 2   carbohydrates         40 non-null     object 
 3   fat                   40 non-null     object 
 4   cholesterol           38 non-null     object 
 5   sodium                40 non-null     object 
 6   cholesterol_level     38 non-null     object 
 7   carbohydrates_level   40 non-null     object 
 8   carbohydrates_score   40 non-null     float64
 9   protien_level         40 non-null     object 
 10  protien_score         40 non-null     float64
 11  fat_level             40 non-null     object 
 12  fat_score             40 non-null     float64
 13  sodium_level          40 non-null     object 
 14  sodium_score          40 non-null     float64
 15  overall_food_quality  40 

In [82]:
final_df = copy.deepcopy(dff)
final_df = final_df.merge(food_quality,on='_id').merge(labour[["tot_hrs", "tot_labour_cost($)","_id"]],on='_id')
final_df.head()

,_id,title,Author,ingredient,direction,nutrition_per_serving,total_ratings,ratings,prep,cook,...,carbohydrates_score,protien_level,protien_score,fat_level,fat_score,sodium_level,sodium_score,overall_food_quality,tot_hrs,tot_labour_cost($)
0,611c269728708657c245a3d9,"Healthier Best Big, Fat, Chewy Chocolate Chip ...",MakeItHealthy,"[2 cups all-purpose flour , ½ teaspoon baking ...","[[Step 1, Preheat oven to 325 degrees F (165 d...",[232 calories; protein 2.6g; carbohydrates 38g...,[5 Ratings],"{'5': 3, '4': 0, '3': 0, '2': 0, '1': 2}",10 mins,15 mins,...,0.0,low,1.0,normal,2.0,good,2.0,excellent,1.416667,14.166667
1,611c273428708657c245a403,Healthier (but still) The Best Rolled Sugar Co...,MakeItHealthy,"[1 ½ cups butter, softened , 1 cup granular su...","[[Step 1, Cream together butter, sweetener, an...",[88 calories; protein 1.5g; carbohydrates 9.3g...,[3 Ratings],"{'5': 1, '4': 2, '3': 0, '2': 0, '1': 0}",20 mins,8 mins,...,1.0,low,1.0,high,0.0,good,2.0,good,1.466667,14.666667
2,611c27d028708657c245a42f,Healthier Guacamole,MakeItHealthy,"[¾ cup crumbled tofu , 2 avocados - peeled and...","[[Step 1, Place tofu and one avocado in a food...",[214 calories; protein 6.4g; carbohydrates 13....,[2 Ratings],"{'5': 1, '4': 0, '3': 0, '2': 1, '1': 0}",10 mins,None,...,1.0,normal,2.0,high,0.0,bad,0.0,good,0.166667,1.666667
3,611c2b1128708657c245a511,Healthier Sweet Potato Pie I,MakeItHealthy,"[1 (1 pound) sweet potato , ½ cup butter, soft...","[[Step 1, Boil sweet potato whole in skin in a...",[316 calories; protein 4.3g; carbohydrates 33....,[3 Ratings],"{'5': 3, '4': 0, '3': 0, '2': 0, '1': 0}",30 mins,50 mins,...,1.0,low,1.0,high,0.0,good,2.0,good,2.333333,23.333333
4,611c2bc428708657c245a542,Healthier Best Chocolate Chip Cookies,MakeItHealthy,"[½ cup butter, softened , ¼ cup sweet potato p...","[[Step 1, Preheat oven to 350 degrees F (175 d...",[249 calories; protein 3.7g; carbohydrates 35....,[3 Ratings],"{'5': 2, '4': 1, '3': 0, '2': 0, '1': 0}",20 mins,10 mins,...,2.0,low,1.0,high,0.0,good,2.0,excellent,1.000000,10.000000


In [83]:
# Give the final df some ML love, but first... do some EDA
final_df.describe()

,weighted_mean_ratings,carbohydrates_score,protien_score,fat_score,sodium_score,tot_hrs,tot_labour_cost($)
count,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000
mean,1.286000,1.325000,1.275000,0.300000,1.300000,1.880417,18.804167
std,1.395534,0.655842,0.640012,0.686873,0.853349,2.405238,24.052379
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,1.666667
25%,0.466667,1.000000,1.000000,0.000000,0.750000,0.645833,6.458333
50%,0.680000,1.000000,1.000000,0.000000,2.000000,1.000000,10.000000
75%,1.580000,2.000000,2.000000,0.000000,2.000000,1.520833,15.208333
max,5.053333,2.000000,2.000000,2.000000,2.000000,10.416667,104.166667


In [84]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _id                    40 non-null     object 
 1   title                  40 non-null     object 
 2   Author                 40 non-null     object 
 3   ingredient             40 non-null     object 
 4   direction              40 non-null     object 
 5   nutrition_per_serving  40 non-null     object 
 6   total_ratings          40 non-null     object 
 7   ratings                40 non-null     object 
 8   prep                   40 non-null     object 
 9   cook                   35 non-null     object 
 10  additional             23 non-null     object 
 11  total                  40 non-null     object 
 12  Servings               40 non-null     object 
 13  Yield                  40 non-null     object 
 14  weighted_mean_ratings  40 non-null     float64
 15  calories

In [92]:
# some useful data are still not in useable form for anlysis.  Try this...
list_to_numeric = ['protein','tot_hrs','tot_labour_cost($)','calories', 'protein','carbohydrates', 'fat', 'cholesterol','sodium']
numeric_dict = {key:'float' for key in list_to_numeric}
final_df = final_df.astype(numeric_dict) 
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _id                    40 non-null     object 
 1   title                  40 non-null     object 
 2   Author                 40 non-null     object 
 3   ingredient             40 non-null     object 
 4   direction              40 non-null     object 
 5   nutrition_per_serving  40 non-null     object 
 6   total_ratings          40 non-null     object 
 7   ratings                40 non-null     object 
 8   prep                   40 non-null     object 
 9   cook                   35 non-null     object 
 10  additional             23 non-null     object 
 11  total                  40 non-null     object 
 12  Servings               40 non-null     object 
 13  Yield                  40 non-null     object 
 14  weighted_mean_ratings  40 non-null     float64
 15  calories

In [93]:
# that looks better. Explore a bit more the numerical data. We will encode the categorical data such as overall food quality  later on
final_df.select_dtypes(exclude='O').head()

,weighted_mean_ratings,calories,protein,carbohydrates,fat,cholesterol,sodium,carbohydrates_score,protien_score,fat_score,sodium_score,tot_hrs,tot_labour_cost($)
0,0.226667,232.0,2.6,38.0,8.7,35.3,46.7,0.0,1.0,2.0,2.0,1.416667,14.166667
1,0.173333,88.0,1.5,9.3,5.0,24.6,53.8,1.0,1.0,0.0,2.0,1.466667,14.666667
2,0.093333,214.0,6.4,13.7,17.1,NaN,595.3,1.0,2.0,0.0,0.0,0.166667,1.666667
3,0.200000,316.0,4.3,33.6,18.6,79.4,245.3,1.0,1.0,0.0,2.0,2.333333,23.333333
4,0.186667,249.0,3.7,35.8,11.4,10.2,95.5,2.0,1.0,0.0,2.0,1.000000,10.000000


In [95]:
# Love sweetviz. Shouout to the developers
# import sweetviz as sv
# sweet_food = sv.analyze(final_df.select_dtypes(exclude='O'),
#             target_feat = 'weighted_mean_ratings',
#             feat_cfg=None,
#             pairwise_analysis= 'auto')
# sweet_food.show_html(
#                 scale=1,
#                 layout='widescreen',
#                 )  

In [ ]:
# Serious questions to ask the data
# On average, do people love good/healthy food?
# Is there a statisitcally significant relationship between star ratings and say labour cost of making the recipe?
# Can we build a nice, unsupervised classification of recipes using ML algorithms?
# What other questions cane we ask on results of unsupervised ML experiments?
# and a lot more questions.